# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [1]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: Error response from daemon: Conflict. The container name "/nginx" is already in use by container "ba99b00f4649c9764627b7bd72aa46e4d349f32be05cad52f24f63fbd9141417". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [ ]:
!curl localhost

#the curl command should fail on our system 

/bin/bash: line 1: curl: command not found


Lets run it on the docker container 

In [3]:
!docker exec -it nginx /bin/sh -c "curl google.com"



the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [4]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [5]:
!docker network ls 

NETWORK ID     NAME      DRIVER    SCOPE
66eb4b20e83c   bridge    bridge    local
af8833370abe   host      host      local
a7115db36a07   none      null      local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [6]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "a7115db36a07d9f743c8e5aaacaf9a7d3e03ad1ce83ff927493f6b3df7661c81",
        "Created": "2025-04-16T00:08:48.057077892Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "ba99b00f4649c9764627b7bd72aa46e4d349f32be05cad52f24f63fbd9141417": {
                "Name": "nginx",
                "EndpointID": "34cc2eb5dfe52640f75bb0a59a43063970aa12c6183ad4b473a7f5e2785a2410",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [7]:
!pip install docker

In [8]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 28.0.4


c:\Users\cadmin\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\cadmin\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\cadmin\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Lets create a quick code to list the containers using the docker client

In [11]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running
mystifying_driscoll - exited


Here you can see the running container and no surprise it is the nginx we started before


In [12]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [13]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

16cfa135112eff67116437e47bb8a5fa8b40c3b1bcdb05eeca1b435ec6ee5ca9


Unable to find image 'mysql:8' locally
8: Pulling from library/mysql
b3dd4e5cd318: Pulling fs layer
10eaa68c5921: Pulling fs layer
d1e5287da18a: Pulling fs layer
cea172a6e83b: Pulling fs layer
aad4571eacc0: Pulling fs layer
182039f5bee3: Pulling fs layer
6c228f7bea3b: Pulling fs layer
07e8f079852f: Pulling fs layer
290ec4c227a4: Pulling fs layer
cb416885cc1f: Pulling fs layer
b3dd4e5cd318: Download complete
10eaa68c5921: Download complete
aad4571eacc0: Download complete
6c228f7bea3b: Download complete
07e8f079852f: Download complete
290ec4c227a4: Download complete
cb416885cc1f: Download complete
cea172a6e83b: Download complete
6c228f7bea3b: Pull complete
cea172a6e83b: Pull complete
290ec4c227a4: Pull complete
10eaa68c5921: Pull complete
cb416885cc1f: Pull complete
07e8f079852f: Pull complete
182039f5bee3: Download complete
d1e5287da18a: Download complete
aad4571eacc0: Pull complete
d1e5287da18a: Pull complete
b3dd4e5cd318: Pull complete
182039f5bee3: Pull complete
Digest: sha256:3a8925

## The following needs to be run in the terminal

In [14]:
!docker run -d --name adminer -p 8080:8080 adminer


d3b7b5a8af9906b5b02cef0fb56014af75dbbea4eace3769f234d1c17c42fb9f


Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer
52e9abf80869: Pulling fs layer
a80d3af78a9d: Pulling fs layer
e434f3dae19e: Pulling fs layer
f3fe38446507: Pulling fs layer
4f4fb700ef54: Pulling fs layer
93b26376816c: Pulling fs layer
115c7a04f6c0: Pulling fs layer
8b847f2809a2: Pulling fs layer
0d55cff53123: Pulling fs layer
c6d0aade0806: Pulling fs layer
88d591373a19: Pulling fs layer
b1220d176a8d: Pulling fs layer
ffeef6b41a56: Pulling fs layer
6d95abe48138: Pulling fs layer
f18232174bc9: Pulling fs layer
dc6d32971fca: Pulling fs layer
a80d3af78a9d: Download complete
f3fe38446507: Download complete
52e9abf80869: Download complete
4f4fb700ef54: Download complete
115c7a04f6c0: Download complete
0d55cff53123: Download complete
c6d0aade0806: Download complete
8b847f2809a2: Download complete
6d95abe48138: Download complete
dc6d32971fca: Download complete
93b26376816c: Download complete
e434f3dae19e: Download complete
b1220d176a8d: Download complete
f1823

Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [16]:
# !docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


In [17]:
# !docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


------------------------------
adminer in network 'bridge' has IP: 172.17.0.3


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [18]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        # print(f"Waiting for {container.name} to stop...")
        # while container.status != 'exited':
        #     pass
        
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


adminer - running
mysql - running
nginx - running
mystifying_driscoll - exited
Removing adminer
Removing mysql
Removing nginx
Removing mystifying_driscoll


## Setting up the network

In [19]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

d51600b17b4de6567912afb65eb9ef0738353624346e571727d8ddee0ffa8d6d


In [20]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "d51600b17b4de6567912afb65eb9ef0738353624346e571727d8ddee0ffa8d6d",
        "Created": "2025-04-19T04:14:38.238270351Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [21]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


d8c58ed66e0e7f792218aaa46f2c816a4be3d0c1222e47125adc955436109a00


In [22]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

38a92c33ffbf8ace558f5eaf8a26166c62877bca55f5ba868f149e9ca16a8e72


In [23]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [23]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.2",
------------------------------


NameError: name 'show_container_ip' is not defined

In [27]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.3",
PING 172.18.0.3 (172.18.0.3) 56(84) bytes of data.
64 bytes from 172.18.0.3: icmp_seq=1 ttl=64 time=0.063 ms

--- 172.18.0.3 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.063/0.063/0.063/0.000 ms
------------------------------


NameError: name 'show_container_ip' is not defined

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [ ]:
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("mysql")
show_container_ip("adminer")



mysql in network 'yourNetwork' has IP: 172.18.0.2
adminer in network 'yourNetwork' has IP: 172.18.0.3


In [ ]:
def container_scanner ():
  

def stop_and_go_container ():
  # client = docker.from_env()
  for container in client.containers.list():
    print(f"Active ID:{container.id}")
    container.stop()
    # print(f"Deactivating:{container.id}")
    container.start()
    # print(f"Now Active IDs: {container.id}")


stop_and_go_container()

# list_running_containers()

Active ID:38a92c33ffbf8ace558f5eaf8a26166c62877bca55f5ba868f149e9ca16a8e72
Active ID:d8c58ed66e0e7f792218aaa46f2c816a4be3d0c1222e47125adc955436109a00



## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/

